In [1]:
!rm -fr r_trader out
!mkdir out input
!git clone https://github.com/abreham-atlaw/r_trader
!cd r_trader &&  git checkout deep-reinforcement.training-experiment-cnn.tmp
!pip install cattrs positional-encodings==6.0.1 dropbox pymongo==4.3.3 dependency-injector==4.41.0

mkdir: cannot create directory ‘input’: File exists
Cloning into 'r_trader'...
remote: Enumerating objects: 17124, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 17124 (delta 66), reused 69 (delta 52), pack-reused 17026 (from 3)
Receiving objects: 100% (17124/17124), 77.92 MiB | 14.39 MiB/s, done.
Resolving deltas: 100% (12282/12282), done.
Branch 'deep-reinforcement.training-experiment-cnn.tmp' set up to track remote branch 'deep-reinforcement.training-experiment-cnn.tmp' from 'origin'.
Switched to a new branch 'deep-reinforcement.training-experiment-cnn.tmp'


In [2]:
import os
KAGGLE_ENV = os.path.exists("/kaggle/working")
REPO_PATH = "/kaggle/working/r_trader" if KAGGLE_ENV else "/content/r_trader"

print(f"KAGGLE ENV: {KAGGLE_ENV}")

import sys
sys.path.append(REPO_PATH)

KAGGLE ENV: False


In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam, SGD, Adagrad
import matplotlib.pyplot as plt

import os
import signal

from core.utils.research.data.load.dataset import BaseDataset
from core.utils.research.training.trainer import Trainer
from core.utils.research.model.model.cnn.model import CNN
from core.utils.research.model.model.linear.model import LinearModel
from lib.utils.torch_utils.model_handler import ModelHandler
from core.utils.research.training.callbacks.checkpoint_callback import CheckpointCallback, StoreCheckpointCallback
from core.utils.research.training.data.repositories.checkpoint_repository import CheckpointRepository
from lib.utils.file_storage import PCloudClient
from core.utils.research.training.data.state import TrainingState
from core import Config
from core.utils.research.training.callbacks.metric_callback import MetricCallback
from core.utils.research.training.data.repositories.metric_repository import MetricRepository, MongoDBMetricRepository
from core.utils.kaggle import FusedManager
from core.di import init_di, ApplicationContainer
from core.utils.research.training.data.metric import MetricsContainer
from core.utils.research.model.layers import Indicators
from core.di import ServiceProvider
from core.utils.kaggle.data_repository import KaggleDataRepository
from core.utils.research.losses import ProximalMaskedLoss

 PID:12955 [2025-01-28 08:24:12.756257]  XLA is not installed. Training using TPU will not be possible.  


In [4]:
def download_data(root, datasets, zip_filename, kernel_mode=True, checksums=None):
    repository = KaggleDataRepository(
        output_path=root,
        zip_filename=zip_filename
    )
    repository.download_multiple(datasets, kernel=kernel_mode, checksums=checksums)
    os.system(f"unzip -d root/")

In [5]:
DATA_ROOT = "/kaggle/input" if KAGGLE_ENV else "/content/input"

DATASETS = [
    f"abrehamatlaw0/rtrader-datapreparer-simsim-cum-0-it-10-0-dataset"
    # for i in range(0,4)
]
CHECKSUMS = [
    '99f3a6b3b3c1482d91a7a8f146ff0787981aaafd51db24c6142e7f60b5a46483',
]
KERNEL_MODE = False
ZIP_FILENAME = "out.zip"
if not KAGGLE_ENV:
    download_data(DATA_ROOT, DATASETS, ZIP_FILENAME, kernel_mode=KERNEL_MODE, checksums=CHECKSUMS)


CONTAINERS = [os.path.join(DATA_ROOT, container) for container in os.listdir(DATA_ROOT)]
DATA_PATHES, TEST_DATA_PATHES = [
    [
        os.path.join(container, "out", type_)
        for container in CONTAINERS
    ]
    for type_ in ["train", "test"]
]

NOTEBOOK_ID = "abrehamalemu/rtrader-training-exp-0-cnn-150-cum-0-it-10-tot"
MODEL_ID = NOTEBOOK_ID.replace("/", "-")

NUM_FILES = None
DATA_CACHE_SIZE = 2
DATALOADER_WORKERS = 4

LR = 1e-4

LOSS_P = 1

BATCH_SIZE = 256
EPOCHS = 100
TIMEOUT = 10*60*60

DTYPE = torch.float32
NP_DTYPE = np.float32

MODEL_URL = None
SAVE_PATH = os.path.abspath("./out/model.zip")
STATE_SAVE_PATH = os.path.abspath("./out/model.zip")

METRIC_REPOSITORY = MongoDBMetricRepository(
    Config.MONGODB_URL,
    MODEL_ID
)

CALLBACKS = [
    StoreCheckpointCallback(
        path=os.path.dirname(SAVE_PATH),
        interval=5,
        fs=ServiceProvider.provide_file_storage(Config.MODEL_IN_PATH),
        active=True
    ),
    MetricCallback(
       METRIC_REPOSITORY
    )
]

 PID:12955 [2025-01-28 08:24:12.836403]  Downloading abrehamatlaw0/rtrader-datapreparer-simsim-cum-0-it-10-0-dataset 
 PID:12955 [2025-01-28 08:24:12.838317]  Downloading to /content/input/abrehamatlaw0-rtrader-datapreparer-simsim-cum-0-it-10-0-dataset 
 PID:12955 [2025-01-28 08:24:12.839994]  Checking pre-downloaded for /content/input/abrehamatlaw0-rtrader-datapreparer-simsim-cum-0-it-10-0-dataset 
 PID:12955 [2025-01-28 08:24:12.841653]  Generating checksum for '/content/input/abrehamatlaw0-rtrader-datapreparer-simsim-cum-0-it-10-0-dataset' 


In [6]:
repository = CheckpointRepository(
    ServiceProvider.provide_file_storage()
)

In [7]:
state_model = repository.get(MODEL_ID)
# state_model = None
if state_model is None:
    raise ValueError(f"Can't Find Model: {MODEL_ID}")

else:
    print("[+]Using loaded model...")
    state, model = state_model
state = TrainingState(
    epoch=0,
    batch=0,
    id=MODEL_ID
)

 PID:12955 [2025-01-28 08:24:15.959003]  Using Storage 0 for abrehamalemu-rtrader-training-exp-0-cnn-150-cum-0-it-10-tot.zip 
[+]Using loaded model...


/content/r_trader/lib/utils/torch_utils/model_handler.py:145: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict_lazy(torch.load(os.path.join(dirname, 'mode

In [8]:
dataset = BaseDataset(
    root_dirs=DATA_PATHES,
    out_dtypes=NP_DTYPE,
    num_files=NUM_FILES
)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=DATALOADER_WORKERS, pin_memory=True)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [9]:
test_dataset = BaseDataset(
    root_dirs=TEST_DATA_PATHES,
    out_dtypes=NP_DTYPE,
    num_files=NUM_FILES
)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=DATALOADER_WORKERS, pin_memory=True)

In [10]:
trainer = Trainer(model, callbacks=CALLBACKS)

 PID:12955 [2025-01-28 08:24:27.020256]  Using device: 1 


In [11]:
trainer.cls_loss_function = ProximalMaskedLoss(
    n=len(Config.AGENT_STATE_CHANGE_DELTA_STATIC_BOUND) + 1 ,
    p=LOSS_P,
    softmax=True,
    device=trainer.device,

)
trainer.reg_loss_function = nn.MSELoss()
trainer.optimizer = Adam(trainer.model.parameters(), lr=LR)

In [12]:
class TimeoutException(Exception):
    pass

def handle_timeout(*args, **kwargs):
    raise TimeoutException()

signal.signal(signal.SIGALRM, handle_timeout)
signal.alarm(TIMEOUT)

0

In [ ]:
try:
    trainer.train(dataloader, val_dataloader=test_dataloader, epochs=EPOCHS, progress=True, progress_interval=1000, state=state, cls_loss_only=False)
except TimeoutException:
    pass

Model Summary
Layer Name							Number of Parameters
layers.0.weight			3840
layers.0.bias			128
layers.1.weight			49152
layers.1.bias			128
layers.2.weight			49152
layers.2.bias			128
layers.3.weight			49152
layers.3.bias			128
ff_block.layers.0.weight			29162496
ff_block.layers.0.bias			256
ff_block.layers.1.weight			65536
ff_block.layers.1.bias			256
ff_block.layers.2.weight			65536
ff_block.layers.2.bias			256
ff_block.layers.3.weight			65536
ff_block.layers.3.bias			256
ff_block.layers.4.weight			110592
ff_block.layers.4.bias			432
Total Params:29622960
[+]Shuffling dataset...


  0%|          | 0/127 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 1 loss: 18.54132843017578(cls: 18.540721893310547, reg: 0.0006072913529351354): 100%|██████████| 127/127 [00:16<00:00,  7.53it/s]

Epoch 1 completed, loss: 17.760488510131836(cls: 17.758596420288086, reg: 0.0018954275874421)


Validation loss: loss: 17.657474517822266(cls: 17.65683364868164, reg: 0.0006374060758389533)
[+]Shuffling dataset...


Epoch 2 loss: 16.928913116455078(cls: 16.92829132080078, reg: 0.0006220544455572963): 100%|██████████| 127/127 [00:16<00:00,  7.51it/s]

...

Validation loss: loss: 39.746315002441406(cls: 39.73957061767578, reg: 0.006742487195879221)
[+]Shuffling dataset...


Epoch 76 loss: 1.8895882368087769(cls: 1.8833880424499512, reg: 0.006200247444212437): 100%|██████████| 127/127 [00:17<00:00,  7.16it/s]

Epoch 76 completed, loss: 2.68506121635437(cls: 2.679381847381592, reg: 0.005678991787135601)


Validation loss: loss: 38.59113311767578(cls: 38.584102630615234, reg: 0.0070134676061570644)
[+]Shuffling dataset...


Epoch 77 loss: 2.482888698577881(cls: 2.4768853187561035, reg: 0.0060034869238734245):  62%|██████▏   | 79/127 [00:10<00:06,  7.42it/s]

In [ ]:
ModelHandler.save(model, SAVE_PATH)

In [ ]:
repository.update(trainer.state, trainer.model)

In [ ]:
metrics = MetricsContainer()
for metric in METRIC_REPOSITORY.get_all():
    metrics.add_metric(metric)

for i in range(3):
    train_losses = [metric.value[i] for metric in metrics.filter_metrics(source=0)]
    val_losses = [metric.value[i] for metric in metrics.filter_metrics(source=1)]
    plt.figure()
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.show()

In [ ]:
for X, y in test_dataloader:
    break
y_hat = model(X.to(trainer.device)).detach().cpu().numpy()

import matplotlib.pyplot as plt
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

def scale(x):
    x = softmax(x)
    x = x / np.max(x)
    return x

for i in range(y_hat.shape[0]):
    plt.figure()
    plt.plot(y[i, :-1])
    plt.plot(scale(y_hat[i, :-1]))


In [ ]:
!rm -fr r_trader